# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import tensorflow as tf
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf =tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

#tf.set_random_seed(seed)
tf.compat.v1.set_random_seed(seed)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

K.set_session(sess)
#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
from keras.datasets import fashion_mnist
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Activation, Dropout, Layer
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import optimizers,initializers,constraints,regularizers
from keras import backend as K
from keras.callbacks import LambdaCallback,ModelCheckpoint
from keras.utils import plot_model

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

import h5py
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse

#--------------------------------------------------------------------------------------------------------------------------------
#Import ourslef defined methods
import sys
sys.path.append(r"./Defined")
import Functions as F

from sklearn.model_selection import train_test_split
import random
import scipy.sparse as sparse
import pandas as pd

Using TensorFlow backend.


# 2. Loading data

In [2]:
Training_samples=6000
Testing_samples=4000

(x_train_, y_train_), (x_test_, y_test_) = fashion_mnist.load_data()
x_train = x_train_.reshape(60000, 28*28).astype('float32')[0:Training_samples] / 255.
x_test__ = x_test_.reshape(10000, 28*28).astype('float32') / 255.

np.random.seed(seed)
x_test__num,_=x_test__.shape
index=np.arange(x_test__num)
np.random.shuffle(index)

x_test=x_test__[index][0:Testing_samples]

y_train=y_train_[0:Training_samples]
y_test=y_test_[index][0:Testing_samples]

features=np.r_[x_train,x_test]
labels=np.r_[y_train,y_test]

print(features.shape,labels.shape)

(10000, 784) (10000,)


# 3.Model

In [3]:
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
rn.seed(seed)
tf.compat.v1.set_random_seed(seed)

#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')
    del dataframe


#--------------------------------------------------------------------------------------------------------------------------------
def top_k_keepWeights_1(p_arr_,p_top_k_,p_ignore_equal=True):
    top_k_idx=p_arr_.argsort()[::-1][0:p_top_k_]
    top_k_value=p_arr_[top_k_idx]
    if np.sum(p_arr_>0)>p_top_k_:
        if p_ignore_equal:
            p_arr_=np.where(p_arr_<top_k_value[-1],0,1)
        else:
            p_arr_=np.where(p_arr_<=top_k_value[-1],0,1)
    else:
        p_arr_=np.where(p_arr_<=0,0,1) 
    return p_arr_


#--------------------------------------------------------------------------------------------------------------------------------
class Feature_Select_Layer(Layer):
    
    def __init__(self, output_dim, **kwargs):
        super(Feature_Select_Layer, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',  
                                      shape=(input_shape[1],),
                                      initializer=initializers.RandomUniform(minval=0.999999, maxval=0.9999999, seed=seed),
                                      trainable=True)
        super(Feature_Select_Layer, self).build(input_shape)
    
    def call(self, x, selection=False,k=50):
        kernel=K.abs(self.kernel)        
        if selection:
            kernel_=K.transpose(kernel)
            kth_largest = tf.math.top_k(kernel_, k=k)[0][-1]
            kernel = tf.where(condition=K.less(kernel,kth_largest),x=K.zeros_like(kernel),y=kernel)        
        return K.dot(x, tf.linalg.tensor_diag(kernel))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

#--------------------------------------------------------------------------------------------------------------------------------
def Fractal_Autoencoder(p_data_feature,\
                        p_feture_number,\
                        p_encoding_dim,\
                        p_learning_rate,\
                        p_loss_weight_1,\
                        p_loss_weight_2,\
                        p_batch_size,\
                        p_epochs,\
                        p_seed_i):
    
    input_img = Input(shape=(p_data_feature,), name='autoencoder_input')

    feature_selection = Feature_Select_Layer(output_dim=p_data_feature,\
                                             input_shape=(p_data_feature,),\
                                             name='feature_selection')

    feature_selection_score=feature_selection(input_img)
    feature_selection_choose=feature_selection(input_img,selection=True,k=p_feture_number)

    encoded = Dense(p_encoding_dim,\
                    activation='linear',\
                    kernel_initializer=initializers.glorot_uniform(seed),\
                    name='autoencoder_hidden_layer')
    
    encoded_score=encoded(feature_selection_score)
    encoded_choose=encoded(feature_selection_choose)
    
    bottleneck_score=encoded_score
    bottleneck_choose=encoded_choose
    
    decoded = Dense(p_data_feature,\
                    activation='linear',\
                    kernel_initializer=initializers.glorot_uniform(seed),\
                    name='autoencoder_output')
    
    decoded_score =decoded(bottleneck_score)
    decoded_choose =decoded(bottleneck_choose)

    latent_encoder_score = Model(input_img, bottleneck_score)
    latent_encoder_choose = Model(input_img, bottleneck_choose)
    feature_selection_output=Model(input_img,feature_selection_choose)
    autoencoder = Model(input_img, [decoded_score,decoded_choose])
    
    autoencoder.compile(loss=['mean_squared_error','mean_squared_error'],\
                        loss_weights=[p_loss_weight_1, p_loss_weight_2],\
                        optimizer=optimizers.Adam(lr=p_learning_rate))
    
    print('Autoencoder Structure-------------------------------------')
    autoencoder.summary()
    
    callback_ES = tf.keras.callbacks.EarlyStopping(monitor='autoencoder_output_loss', patience=20)
    
    x_train_s,x_test_s,_,_= train_test_split(features,labels,test_size=0.1,random_state=p_seed_i)

    print('Train', x_train_s.shape)
    print('Test', x_test_s.shape)
    
    model_history=autoencoder.fit(x_train_s,[x_train_s,x_train_s],batch_size=p_batch_size,epochs=p_epochs,verbose=1,callbacks=[callback_ES])

    #--------------------------Key features display--------------------------------------------------------------
    key_features=top_k_keepWeights_1(autoencoder.get_layer(index=1).get_weights()[0],p_feture_number)
    
    return key_features

## 3.1 Structure and paramter testing

In [4]:
epochs_number=200
batch_size_value=256

---
### 3.1.1 Fractal Autoencoder
---

In [ ]:
loss_weight_1=0.0078125
key_feture_number=10
number_seed=50

for p_seed_i in np.arange(number_seed):
    key_features_vector=Fractal_Autoencoder(p_data_feature=784,\
                                            p_feture_number=key_feture_number,\
                                            p_encoding_dim=key_feture_number,\
                                            p_learning_rate= 1E-3,\
                                            p_loss_weight_1=loss_weight_1,\
                                            p_loss_weight_2=1,\
                                            p_batch_size=batch_size_value,\
                                            p_epochs=epochs_number,\
                                            p_seed_i=p_seed_i)
    
    write_to_csv(key_features_vector.reshape(1,len(key_features_vector)),"./log/key_features"+str(key_feture_number)+".csv")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Autoencoder Structure-------------------------------------
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
autoencoder_input (InputLayer)  (None, 784)          0                                            
__________________________________________________________________________________________________
feature_selection (Feature_Sele (None, 784)          784         autoencoder_input[0][0]          
                                                                 autoencoder_input[0][0]          
__________________________________________________________________________________________________
autoencoder_hidden_layer (Dense (None, 10)           785